In [1]:
## INITIALIZE ##
from dotenv import load_dotenv
import os
from langgraph_app import app_graph
from IPython.display import Markdown, display, HTML
import pprint

# Load environment variables from .env file in the current directory
load_dotenv()

# Enable automatic reloading
%load_ext autoreload
%autoreload 2

In [2]:
## SELECT FILES ##
import os
import glob
import warnings

# Suppress tkinter warnings on macOS
warnings.filterwarnings("ignore", category=DeprecationWarning)

try:
    import tkinter as tk
    from tkinter import filedialog
    
    # Suppress macOS console warnings
    import sys
    from contextlib import redirect_stderr
    from io import StringIO
    
    # Create a root window and hide it
    root = tk.Tk()
    root.withdraw()  # Hide the main window
    root.attributes('-topmost', True)  # Bring dialog to front
    
    # Set the initial directory to the images folder
    initial_dir = os.path.abspath("images")
    
    print("Opening file selection dialog...")
    print("You can select multiple files by holding Cmd (Mac) or Ctrl (Windows/Linux)")
    
    # Capture stderr to suppress warnings
    f = StringIO()
    with redirect_stderr(f):
        # Open file dialog for multiple image files
        selected_images = filedialog.askopenfilenames(
            title="Select Discharge Summary Images",
            initialdir=initial_dir,
            filetypes=[
                ("Image files", "*.jpg *.jpeg *.png *.gif *.bmp *.tiff"),
                ("JPEG files", "*.jpg *.jpeg"),
                ("PNG files", "*.png"),
                ("All files", "*.*")
            ]
        )
    
    # Clean up the tkinter root window
    root.destroy()
    
    # Convert to list and make paths relative to current directory
    selected_images = list(selected_images)
    if selected_images:
        # Convert absolute paths to relative paths if they're in the images directory
        relative_images = []
        current_dir = os.getcwd()
        for img_path in selected_images:
            try:
                rel_path = os.path.relpath(img_path, current_dir)
                relative_images.append(rel_path)
            except ValueError:
                # If relpath fails, use absolute path
                relative_images.append(img_path)
        
        selected_images = relative_images
        
        print(f"\nSelected {len(selected_images)} files:")
        for file in selected_images:
            print(f"  - {file}")
        print("\nYou can now run the OCR cell with these selected files.")
    else:
        print("\nNo files selected.")
        selected_images = []

except ImportError:
    print("tkinter not available. Using fallback file selection...")
    # Fallback to showing available files
    image_files = glob.glob("images/*.jpeg") + glob.glob("images/*.jpg") + glob.glob("images/*.png")
    image_files = [f for f in image_files if not f.endswith('.DS_Store')]
    image_files.sort()
    
    print("Available image files:")
    for i, file in enumerate(image_files, 1):
        print(f"{i}. {file}")
    
    # Use all available files as default
    selected_images = image_files
    print(f"\nUsing all {len(selected_images)} available files:")
    for file in selected_images:
        print(f"  - {file}")

Opening file selection dialog...
You can select multiple files by holding Cmd (Mac) or Ctrl (Windows/Linux)


2025-08-26 13:35:51.979 Python[39556:3283086] The class 'NSOpenPanel' overrides the method identifier.  This method is implemented by class 'NSWindow'



Selected 4 files:
  - images/3.4.jpeg
  - images/3.1.jpeg
  - images/3.2.jpeg
  - images/3.3.jpeg

You can now run the OCR cell with these selected files.


In [ ]:
## OCR ##
# Use the selected images from the file selection above
if 'selected_images' in globals() and selected_images:
    images = selected_images
    print(f"Processing {len(images)} selected files:")
    for img in images:
        print(f"  - {img}")
else:
    # Fallback to default files if none selected
    images = ["images/3.1.jpeg", "images/3.2.jpeg", "images/3.3.jpeg", "images/3.4.jpeg"]
    print("No files selected, using default files:")
    for img in images:
        print(f"  - {img}")

print("\nRunning OCR...")
markdown = app_graph.run_node("OCR", images, "gpt-4o")

# Clean up markdown text - remove code block wrappers if present
if markdown.startswith("```markdown\n"):
    markdown = markdown[12:]  # Remove "```markdown\n"
if markdown.endswith("\n```"):
    markdown = markdown[:-4]  # Remove "\n```"
elif markdown.endswith("```"):
    markdown = markdown[:-3]  # Remove "```"

# Also handle case where it starts with just ```
if markdown.startswith("```\n"):
    markdown = markdown[4:]  # Remove "```\n"

# Save the cleaned markdown to a file
with open("discharge.md", "w", encoding="utf-8") as f:
    f.write(markdown)
print("Markdown saved to discharge.md")

# Launch the markdown file in the default application
import subprocess
import sys
try:
    if sys.platform == "darwin":  # macOS
        subprocess.run(["open", "discharge.md"], check=True)
        print("Opened discharge.md in default application")
    elif sys.platform == "win32":  # Windows
        subprocess.run(["start", "discharge.md"], shell=True, check=True)
        print("Opened discharge.md in default application")
    else:  # Linux
        subprocess.run(["xdg-open", "discharge.md"], check=True)
        print("Opened discharge.md in default application")
except subprocess.CalledProcessError as e:
    print(f"Could not open discharge.md automatically: {e}")
except FileNotFoundError:
    print("Could not find system command to open discharge.md")

# Display the markdown in the notebook
display(Markdown(markdown))

Processing 4 selected files:
  - images/3.4.jpeg
  - images/3.1.jpeg
  - images/3.2.jpeg
  - images/3.3.jpeg

Running OCR...
=== OCR Node Output (Model: gpt-4o) ===
```markdown
# SHUSRUSHA
A UNIT OF SHUSRUSHA NURSING HOME PVT. LTD.
P-290, C.I.T. Scheme VIM, Swami Swarupananda Sarani, Kolkata - 700 054  
Phone: 2362 8863, 2362 8430, 2364 8910

## DISCHARGE / TRANSFER / DISCHARGE AGAINST MEDICAL ADVISE CERTIFICATE

**Name:** Mrs. Bina Jaicwal  
**UHID No.:** 191020039341  
**Bed No.:** 109  
**Sex:** Female  
**Age:** 64 yrs  
**Address:** 40 Mr. Rohit Jaicwal, Raj, Bholanath Sarani Road, Teghoria, Rajarhat, Gopalpur, Baguihati 700157  

**Date of Admission:** 24/8/15  
**Time of Admission:** 10:00 AM  
**Date of Discharge:** 26/8/15  
**Time of Discharge:**  

I, Dr. Sunil Tibrewal, hereby certify that the above-mentioned patient was under my treatment in this Nursing Home.

### REASONS FOR ADMISSION : PRESENTING COMPLAINTS
- Vomiting
- Abdominal distension
- Swelling

### FINAL DIAGNOS

# SHUSRUSHA
A UNIT OF SHUSRUSHA NURSING HOME PVT. LTD.
P-290, C.I.T. Scheme VIM, Swami Swarupananda Sarani, Kolkata - 700 054  
Phone: 2362 8863, 2362 8430, 2364 8910

## DISCHARGE / TRANSFER / DISCHARGE AGAINST MEDICAL ADVISE CERTIFICATE

**Name:** Mrs. Bina Jaicwal  
**UHID No.:** 191020039341  
**Bed No.:** 109  
**Sex:** Female  
**Age:** 64 yrs  
**Address:** 40 Mr. Rohit Jaicwal, Raj, Bholanath Sarani Road, Teghoria, Rajarhat, Gopalpur, Baguihati 700157  

**Date of Admission:** 24/8/15  
**Time of Admission:** 10:00 AM  
**Date of Discharge:** 26/8/15  
**Time of Discharge:**  

I, Dr. Sunil Tibrewal, hereby certify that the above-mentioned patient was under my treatment in this Nursing Home.

### REASONS FOR ADMISSION : PRESENTING COMPLAINTS
- Vomiting
- Abdominal distension
- Swelling

### FINAL DIAGNOSIS
- Subacute intestinal obstruction
- HTN
- Hypothyroid
- Hepatitis C Positive

**ICD 10 CODE:**  

**H/O ALCOHOLISM / TOBACCO / OTHERS**  
MLC Y / N: Yes

---

### SIGNIFICANT PAST MEDICAL AND SURGICAL AND ALLERGY OR FAMILY HISTORY
- H/O TAH & BSO -> Hernia & R nephrectomy

### SUMMARY OF KEY INVESTIGATION REPORT TO ARRIVE AT THE DIAGNOSIS
- CECT: Collapsed long segment ileum. Sol adheres to R suprarenal gland.

### SUMMARY OF TREATMENT [NAME OF THE PROCEDURES/SURGERIES (IF ANY)]
- Referred Doctors Name:  
- Specialist Name:  

### NOTES ON MEDICAL MANAGEMENT
- Received Cap. Vizylac, Buch, Zedott, ORS, Rantac, Volaprea retard.

**Medical Treatment**  
- Blood Transfusion:  
  - PRBC: Yes / No  
  - FFP: Yes / No  
  - Human Albumin: Yes / No  
  - TPN: Yes / No  
  - Pethidine/Fentanyl/Morphin: Yes / No  

- Antibiotic Used:  
- Anti Emetic Used:  
- PPI:  

**Other Supportive Medicines:**  

---

### SURGICAL MANAGEMENT : NAME OF THE SURGERY / PROCEDURE
- Patient responded to conservative treatment.

### COMPLICATIONS IN THE COURSE OF HOSPITALISATION

### PATIENT CONDITION AT THE TIME OF DISCHARGE
- Ambulatory: ☑️  
- Non Ambulatory: ☐  
- With Oxygen: ☐  
- With BIPAP/CPAP Support: ☐  
- Any Other Support:  

### WHEN & HOW TO OBTAIN URGENT CARE & FOLLOW UP ADVICE
- Fever: ☑️  
- Wound Oozing: ☑️  
- SOB: ☐  
- Surgical Site Pain: ☐  
- Urine Retention: ☐  
- Bleeding: ☐  
- Any Other:  

**FOLLOW UP ADVICE**

---

## ADVICE ON DISCHARGE

1. Ambulatory / Semisolid diet
2. Tab Amlod 5 - 1 tab od after BF x cont
3. Tab Thyroxin 100 - 1 tab od before BF x cont
4. Tab Telma 40 - 1 tab od after dinner x cont
5. Tab Rantac 40 - odac x cont 10 days
6. Tab Ondem MD 4 mg - 6 od x 3 days
7. Syp. Mucaine 25 ml syrup t 1 cons  
   Aluminium citrate at bedtime x cont 7 days

8. After 7 days  
   Review with:
   - a) Dr. P. Boddar - Liver specialist for Hepatitis C (HS DM)
   - b) PET CT scan for evaluation of sol adheres to R suprarenal gland.
   - c) Review for FE after 2 weeks  
     attend earlier / ER SOS

**Signature of the Patient / Attendant**  
**Signature of the Treating Doctor / RMO**  
**Reg. No.**  
**Date:** 26/8/2015
```


In [ ]:
## CONFIGURE PROCESSING STEPS ##
# Configure which steps to run after OCR
run_extract_diagnoses = True  # Set to False to skip diagnoses extraction
run_extract_medications = True  # Set to False to skip medication extraction
run_fix_medications = True  # Set to False to skip PharmeEasy integration
run_generate_summary = True  # Set to False to skip HTML summary generation

print("Processing Steps Configuration:")
print(f"📊 Extract Diagnoses: {'✅ Enabled' if run_extract_diagnoses else '❌ Disabled'}")
print(f"💊 Extract Medications: {'✅ Enabled' if run_extract_medications else '❌ Disabled'}")
print(f"🔗 Fix Medications (PharmeEasy): {'✅ Enabled' if run_fix_medications else '❌ Disabled'}")
print(f"📋 Generate Summary: {'✅ Enabled' if run_generate_summary else '❌ Disabled'}")
print()
print("💡 Tip: Change the variables above to True/False to enable/disable steps")
print("📝 Note: Some steps depend on previous ones (e.g., Fix Medications needs Extract Medications)")

In [ ]:
## EXTRACT DIAGNOSES ##
if 'run_extract_diagnoses' in globals() and run_extract_diagnoses:
    print("🩺 Extracting diagnoses...")
    diagnoses = app_graph.run_node("ExtractDiagnoses", model="gpt-4o-mini")
    print("✅ Diagnoses extraction completed")
else:
    print("⏭️ Skipping diagnoses extraction (disabled in configuration)")
    diagnoses = None

In [ ]:
## EXTRACT MEDICATIONS ##
if 'run_extract_medications' in globals() and run_extract_medications:
    print("💊 Extracting medications...")
    medications = app_graph.run_node("ExtractMedications", model="gpt-4o-mini")
    print("✅ Medications extraction completed")
else:
    print("⏭️ Skipping medications extraction (disabled in configuration)")
    medications = None

In [ ]:
## FIX MEDICATIONS ##
if 'run_fix_medications' in globals() and run_fix_medications:
    # Check if medications were extracted first
    if 'medications' in globals() and medications is not None:
        print("🔗 Matching medications with PharmeEasy...")
        fixed_medications = app_graph.run_node("FixMedications", model="gpt-4o")
        print("✅ Medication fixing completed")
    else:
        print("⚠️ Cannot fix medications - medications extraction was skipped or failed")
        print("💡 Enable 'run_extract_medications' to use this step")
        fixed_medications = None
else:
    print("⏭️ Skipping medication fixing (disabled in configuration)")
    fixed_medications = None

In [ ]:
## GENERATE SUMMARY ##
if 'run_generate_summary' in globals() and run_generate_summary:
    print("📋 Generating interactive HTML summary...")
    html_summary = app_graph.run_node("AddSummaryPills")
    
    # Save the HTML summary to a file
    with open("summary.html", "w", encoding="utf-8") as f:
        f.write(html_summary)
    print("Summary saved to summary.html")
    
    # Launch the HTML file in the default application
    import subprocess
    import sys
    try:
        if sys.platform == "darwin":  # macOS
            subprocess.run(["open", "summary.html"], check=True)
            print("Opened summary.html in default browser")
        elif sys.platform == "win32":  # Windows
            subprocess.run(["start", "summary.html"], shell=True, check=True)
            print("Opened summary.html in default browser")
        else:  # Linux
            subprocess.run(["xdg-open", "summary.html"], check=True)
            print("Opened summary.html in default browser")
    except subprocess.CalledProcessError as e:
        print(f"Could not open summary.html automatically: {e}")
    except FileNotFoundError:
        print("Could not find system command to open summary.html")
    
    # Display the HTML in the notebook
    display(HTML(html_summary))
    print("✅ HTML summary generation completed")
else:
    print("⏭️ Skipping HTML summary generation (disabled in configuration)")
    html_summary = None